In [1]:
import os
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import numpy as np
from PIL import Image
import torch
import pandas as pd

device = "cuda:2"
torch.cuda.set_device(device)


def get_files_recursively(folder_path):
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.split(".")[-1] in ["jpg", "png", "jpeg"]:
                file_list.append(os.path.join(root, file))
    return file_list


dir_path = "/data/noah/inference/data_filtering"
img_list = list()

all_files = get_files_recursively(dir_path)
img_list = all_files
# img_model = SentenceTransformer("clip-ViT-B-32", device=device)
# embeddings = np.zeros((len(img_list), 512))
# labels = list()
# paths = list()

# for i, img_path in enumerate(tqdm(img_list)):
#     img = Image.open(img_path).convert("RGB")
#     embedding = img_model.encode(img)
#     embedding = np.array(embedding, dtype=float)
#     embeddings[i] = embedding
#     # embeddings.append(list(embedding))

#     paths.append(img_path)
#     # labels.append(img_path.split('_')[-1][:-4])

#     is_real = True if os.path.basename(img_path).startswith("real") else False
#     if is_real:
#         labels.append("real")
#     else:
#         labels.append("synthetic")

# embedding_result = TSNE(n_components=2).fit_transform(embeddings)

# df_embeddings = pd.DataFrame(embedding_result)
# df_embeddings = df_embeddings.rename(columns={0: "x", 1: "y"})
# df_embeddings = df_embeddings.assign(label=labels)
# df_embeddings = df_embeddings.assign(path=paths)

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_embeddings,
    x="x",
    y="y",
    color="label",
    labels={"color": "label"},
    hover_data=["path"],
    title="Synthetic Embedding",
)

fig.show()

In [ ]:
distances = []  # {'distance', 'syn_name'}
points_syn = []  # {'embed', 'syn_name'}
points_real = []  # {'embed', 'real_name'}

for idx, emb in enumerate(embedding_result):
    img_name = os.path.basename(paths[idx])

    if img_name.startswith("real"):
        points_real.append({"embed": emb, "real_name": img_name})
    else:
        points_syn.append({"embed": emb, "syn_name": img_name})

import math

for ps in points_syn:
    for pr in points_real:
        if ps["syn_name"].split("_")[-1] == pr["real_name"].split("_")[-1]:
            dis = math.sqrt((ps["embed"][0] - pr["embed"][0]) ** 2 + (ps["embed"][1] - pr["embed"][1]) ** 2)
            distances.append({"distance": dis, "syn_name": ps["syn_name"]})
            break

distances = sorted(distances, key=lambda x: x["distance"])

print(distances)

In [ ]:
import matplotlib.pyplot as plt

target_len = 20
top_dis = distances[:target_len]
top_images = []

for t in top_dis:
    img_path = os.path.join(dir_path, t["syn_name"])
    img = Image.open(img_path).convert("RGB")
    top_images.append(img)

plt.figure(figsize=(50, 35))

for i in range(target_len):
    plt.subplot(4, 5, i + 1)
    plt.imshow(top_images[i])
    plt.title(top_dis[i]["syn_name"])
    plt.axis("off")

plt.show()

In [ ]:
low_dis = distances[-target_len:]
low_images = []

for l in low_dis:
    img_path = os.path.join(dir_path, l["syn_name"])
    img = Image.open(img_path).convert("RGB")
    low_images.append(img)

plt.figure(figsize=(50, 35))

for i in range(target_len):
    plt.subplot(4, 5, i + 1)
    plt.imshow(low_images[i])
    plt.title(top_dis[i]["syn_name"])
    plt.axis("off")

plt.show()